In [5]:

# --- Cell 1: Setup and Requirements ---

!pip install pandas numpy scikit-learn xgboost requests joblib

import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score
import joblib
import time
from datetime import datetime
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.7.2 xgboost pandas numpy joblib requests

# --- Configuration (UPDATE THIS!) ---
# You need a key from OpenWeatherMap or a similar service to fetch real-time data.
WEATHER_API_KEY = "5197fc88f5f846ee7566eb28d403c91f"

# Added from app.py: HERE API key and placeholder check
HERE_API_KEY = "9JI9eOC0auXHPTmtQ5SohrGPp4WjOaq90TRCjfa-Czw"
PLACEHOLDER_CHECK = "PASTE_YOUR_API_KEY_HERE"

THRESHOLD_MIN = 10  # Delivery is "Late" if actual time > estimated time + THRESHOLD_MIN

def setup_environment():
    """Confirms environment setup."""
    print("Environment setup check completed. Required libraries are installed.")

# Run setup
setup_environment()


# --- Cell 2: Helper Functions ---

def haversine(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance in km between two points on the earth."""
    R = 6371  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def fetch_realtime_weather(latitude, longitude, api_key):
    """Fetches real-time weather data for a given location using OpenWeatherMap API.
    Returns (temp, weather_main, wind_speed). Uses fallback on failure."""
    if not api_key or api_key.strip() == "" or api_key == "YOUR_OPENWEATHERMAP_API_KEY":
        print("Warning: Weather API Key missing or placeholder. Using fallback weather values.")
        return 25.0, 'Clear', 5.0  # Return fallback values for simulation

    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={api_key}&units=metric"
        response = requests.get(url, timeout=6)
        response.raise_for_status()
        data = response.json()

        temp = data['main']['temp']
        weather_main = data['weather'][0]['main']
        wind_speed = data['wind'].get('speed', np.nan)

        print(f"Weather fetched successfully: {weather_main}, {temp}°C")
        return temp, weather_main, wind_speed

    except requests.exceptions.HTTPError as e:
        # If auth error or other HTTP errors occur, fallback
        try:
            status = response.status_code
            print(f"Weather API HTTP Error ({status}): {e}. Using fallback.")
        except Exception:
            print(f"Weather API HTTP Error: {e}. Using fallback.")
        return np.nan, 'Unknown', np.nan

    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}. Using fallback.")
        return np.nan, 'Unknown', np.nan


def fetch_coordinates(location_name, api_key):
    """Forward geocoding: returns (lat, lon, display_name) using OpenWeatherMap geocoding endpoint.
    If not possible, returns (None, None, None)."""
    if not api_key or not location_name:
        return None, None, None
    try:
        url = f"http://api.openweathermap.org/geo/1.0/direct?q={location_name}&limit=1&appid={api_key}"
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        data = response.json()
        if data:
            loc = data[0]
            lat = loc.get('lat')
            lon = loc.get('lon')
            display_name = f"{loc.get('name', 'N/A')}, {loc.get('country', 'N/A')}"
            return lat, lon, display_name
        else:
            print(f"Warning: Could not find coordinates for: {location_name}")
            return None, None, None
    except requests.exceptions.RequestException as e:
        print(f"Geocoding Error: {e}")
        return None, None, None


def fetch_live_traffic_time(rest_lat, rest_lon, del_lat, del_lon, api_key):
    """
    Call HERE Routing API with traffic enabled (if API key present).
    Returns (estimated_travel_time_traffic_adjusted_min, base_travel_time_min) in minutes.
    If API unavailable or key missing/placeholder, returns (None, None) to indicate fallback simulation should be used.
    """
    if not api_key or api_key == PLACEHOLDER_CHECK:
        # No real traffic available; caller should fallback to simulation
        return None, None

    try:
        url = "https://router.hereapi.com/v8/routes"
        origin_coords = f"{rest_lat},{rest_lon}"
        destination_coords = f"{del_lat},{del_lon}"
        params = {
            'transportMode': 'car',
            'origin': origin_coords,
            'destination': destination_coords,
            'routingMode': 'fast',
            'trafficMode': 'realtime',
            'return': 'summary',
            'apiKey': api_key
        }
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data.get('routes') and data['routes'][0].get('sections'):
            summary = data['routes'][0]['sections'][0]['summary']
            traffic_duration_sec = summary.get('duration')
            base_duration_sec = summary.get('baseDuration')
            if traffic_duration_sec is None or base_duration_sec is None:
                print("HERE API response missing duration data. Falling back to simulation.")
                return None, None
            traffic_duration_min = traffic_duration_sec / 60.0
            base_travel_time_min = base_duration_sec / 60.0
            print("Live traffic data successfully retrieved from HERE API.")
            return traffic_duration_min, base_travel_time_min
        else:
            print("HERE API failed to find route. Falling back to simulation.")
            return None, None

    except requests.exceptions.RequestException as e:
        print(f"HERE API Connection Error: {e}. Falling back to simulation.")
        return None, None
    except Exception as e:
        print(f"Error parsing HERE response: {e}. Falling back to simulation.")
        return None, None


# --- Cell 3: Dataset Creation and Feature Engineering ---

def get_or_create_dataset():
    """Simulates loading and initial feature engineering on a historical dataset."""
    print("... Loading or creating simulated historical dataset ...")

    # 1. Simulate Historical Data
    np.random.seed(42)  # For reproducibility
    data = {
        'Order_ID': range(500),  # Increased sample size
        'Restaurant_lat': np.random.uniform(28.5, 28.7, 500),
        'Restaurant_lon': np.random.uniform(77.1, 77.3, 500),
        'Delivery_lat': np.random.uniform(28.5, 28.7, 500),
        'Delivery_lon': np.random.uniform(77.1, 77.3, 500),
        'Order_Placed_Time': pd.to_datetime(pd.date_range('2025-01-01', periods=500, freq='4H')),
        'Initial_Estimate_Min': np.random.randint(25, 45, 500),
        'Actual_Delivery_Time_Min': np.random.randint(20, 70, 500),
        'preparation_time_min': np.random.randint(10, 30, 500),
        'restaurant_rating': np.random.uniform(3.0, 5.0, 500).round(1),
        'delivery_person_rating': np.random.uniform(4.0, 5.0, 500).round(1),
        'Road_Traffic_Density': np.random.choice(['Low', 'Medium', 'High', 'Jam'], 500, p=[0.4, 0.3, 0.2, 0.1]),
        'Weather_Condition': np.random.choice(['Clear', 'Rainy', 'Foggy', 'Stormy'], 500, p=[0.7, 0.2, 0.05, 0.05])
    }
    df = pd.DataFrame(data)

    # 2. Feature Engineering
    df['delivery_distance_km'] = haversine(
        df['Restaurant_lat'], df['Restaurant_lon'],
        df['Delivery_lat'], df['Delivery_lon']
    )

    # Time-based features
    df['order_hour'] = df['Order_Placed_Time'].dt.hour
    df['day_of_week'] = df['Order_Placed_Time'].dt.day_name()

    # Cyclic features for hour
    df['sin_hour'] = np.sin(2 * np.pi * df['order_hour'] / 24)
    df['cos_hour'] = np.cos(2 * np.pi * df['order_hour'] / 24)

    # Target Variable
    df['is_late'] = np.where(
        df['Actual_Delivery_Time_Min'] > (df['Initial_Estimate_Min'] + THRESHOLD_MIN),
        1,
        0
    )

    # Add simulated weather for training data (as historical data won't use the live API)
    df['current_temp_c'] = np.random.uniform(15, 35, 500)

    final_features = [
        'delivery_distance_km', 'preparation_time_min', 'restaurant_rating',
        'delivery_person_rating', 'Road_Traffic_Density', 'Weather_Condition',
        'sin_hour', 'cos_hour'
    ]

    df_train = df[final_features + ['current_temp_c', 'is_late']].copy()

    print(f"Dataset created with {len(df_train)} samples. Target variable 'is_late' distribution:")
    print(df_train['is_late'].value_counts(normalize=True))
    return df_train, final_features


# --- Cell 4: Preprocessing Functions ---

def create_preprocessing_pipeline(feature_list):
    """Creates a scikit-learn ColumnTransformer for preprocessing."""

    numeric_features = [
        'delivery_distance_km', 'preparation_time_min', 'restaurant_rating',
        'delivery_person_rating', 'current_temp_c', 'sin_hour', 'cos_hour'
    ]

    categorical_features = [
        'Road_Traffic_Density', 'Weather_Condition'
    ]

    # Create preprocessing steps
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Combine transformers
    # Only include features present in feature_list (plus current_temp_c)
    num_cols = [f for f in numeric_features if (f in feature_list) or (f == 'current_temp_c')]
    cat_cols = [f for f in categorical_features if f in feature_list]

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_cols),
            ('cat', categorical_transformer, cat_cols)
        ],
        remainder='passthrough'
    )

    print("Preprocessing pipeline (Scaling + One-Hot Encoding) created.")
    return preprocessor  # FIX: return the actual preprocessor object


def perform_preprocessing(df_train, feature_list, preprocessor):
    """Splits data and prepares for the full pipeline."""

    # Include 'current_temp_c' in X for training as it's a numeric feature
    X = df_train[feature_list + ['current_temp_c']].copy()
    y = df_train['is_late']

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    print(f"Data split: Train size: {len(X_train)}, Test size: {len(X_test)}")
    return X_train, X_test, y_train, y_test


# --- Cell 5: Model Training, Evaluation, and Saving ---

def train_and_evaluate_model(X_train, X_test, y_train, y_test, preprocessor):
    """Defines, trains, and evaluates the final ML pipeline."""

    # Model Choice: XGBoost Hyperparameters
    XGB_PARAMS = {
        'objective': 'binary:logistic',
        'n_estimators': 300,
        'learning_rate': 0.05,
        'max_depth': 5,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'random_state': 42,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }

    model = XGBClassifier(**XGB_PARAMS)

    # Create the full ML pipeline
    full_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    print("... Training XGBoost model ...")
    start_time = time.time()
    full_pipeline.fit(X_train, y_train)
    end_time = time.time()
    print(f"Training completed in {end_time - start_time:.2f} seconds.")

    # Evaluation
    y_pred = full_pipeline.predict(X_test)
    y_proba = full_pipeline.predict_proba(X_test)[:, 1]

    print("\n--- Model Evaluation (Test Set) ---")
    print(classification_report(y_test, y_pred))
    print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")

    # Save the model
    joblib.dump(full_pipeline, 'late_delivery_predictor_model.pkl')
    print("Model saved as 'late_delivery_predictor_model.pkl'.")

    return full_pipeline


# --- Cell 6: Real-Time Dashboard (Conceptual & Demonstration) ---

def create_real_time_dashboard(model_pipeline, features):
    """
    Provides the conceptual code for the Streamlit dashboard and demonstrates a real-time prediction using the saved model.
    This function performs a real-time prediction demonstration (not a full Streamlit app).
    """

    print("\n--- Conceptual Streamlit Dashboard Code (omitted) ---")
    print("To run the full dashboard, save the Streamlit app separately (app.py).")
    print("\n--- Real-Time Prediction Demonstration ---")

    # 1. Define a New Order (Input Data)
    NEW_ORDER_DATA = {
        'Restaurant_lat': 28.60,
        'Restaurant_lon': 77.15,
        'Delivery_lat': 28.70,
        'Delivery_lon': 77.25,
        'preparation_time_min': 25,
        'restaurant_rating': 4.2,
        'delivery_person_rating': 4.9,
    }

    # 2. Feature Engineering
    current_time = pd.Timestamp.now(tz='Asia/Kolkata')
    order_hour = current_time.hour

    delivery_distance_km = haversine(
        NEW_ORDER_DATA['Restaurant_lat'], NEW_ORDER_DATA['Restaurant_lon'],
        NEW_ORDER_DATA['Delivery_lat'], NEW_ORDER_DATA['Delivery_lon']
    )

    # Simulate Real-time Dynamic Features
    temp, weather_main, wind_speed = fetch_realtime_weather(
        NEW_ORDER_DATA['Delivery_lat'],
        NEW_ORDER_DATA['Delivery_lon'],
        WEATHER_API_KEY
    )

    # Traffic Density Simulation based on time (for demonstration)
    # Use multipliers and labels adopted from app.py
    if 17 <= order_hour <= 21:
        traffic_multiplier_sim = 1.67
        traffic_label_sim = 'Jam'
    elif 12 <= order_hour <= 14:
        traffic_multiplier_sim = 1.33
        traffic_label_sim = 'High'
    elif 8 <= order_hour <= 10:
        traffic_multiplier_sim = 1.18
        traffic_label_sim = 'Medium'
    else:
        traffic_multiplier_sim = 1.0
        traffic_label_sim = 'Low'

    sin_hour = np.sin(2 * np.pi * order_hour / 24)
    cos_hour = np.cos(2 * np.pi * order_hour / 24)

    # 3. Try to get live traffic via HERE API. If unavailable, fallback to simulation.
    api_result = fetch_live_traffic_time(
        NEW_ORDER_DATA['Restaurant_lat'], NEW_ORDER_DATA['Restaurant_lon'],
        NEW_ORDER_DATA['Delivery_lat'], NEW_ORDER_DATA['Delivery_lon'],
        HERE_API_KEY
    )
    estimated_travel_time_traffic_adjusted, base_travel_time_min_api = api_result

    BASE_SPEED_KM_PER_MIN = 0.5  # 30 km/h baseline

    if estimated_travel_time_traffic_adjusted is None:
        base_travel_time_min = delivery_distance_km / BASE_SPEED_KM_PER_MIN
        estimated_travel_time_traffic_adjusted = base_travel_time_min * traffic_multiplier_sim
        traffic_density = traffic_label_sim
    else:
        traffic_ratio = estimated_travel_time_traffic_adjusted / base_travel_time_min_api
        if traffic_ratio >= 1.5:
            traffic_density = 'Jam'
        elif traffic_ratio >= 1.25:
            traffic_density = 'High'
        elif traffic_ratio >= 1.05:
            traffic_density = 'Medium'
        else:
            traffic_density = 'Low'

    # --- Prediction DataFrame Construction (match ordering and raw categorical columns) ---
    input_data_final = pd.DataFrame({
        'delivery_distance_km': [delivery_distance_km],
        'preparation_time_min': [NEW_ORDER_DATA['preparation_time_min']],
        'restaurant_rating': [NEW_ORDER_DATA['restaurant_rating']],
        'delivery_person_rating': [NEW_ORDER_DATA['delivery_person_rating']],
        'Road_Traffic_Density': [traffic_density],      # categorical
        'Weather_Condition': [weather_main],            # categorical
        'sin_hour': [sin_hour],
        'cos_hour': [cos_hour],
        'current_temp_c': [temp if not np.isnan(temp) else 25.0]
    })

    # 4. Predict
    try:
        prediction_proba = model_pipeline.predict_proba(input_data_final)[:, 1][0] * 100
    except Exception as e:
        print(f"Prediction Error: {e}")
        prediction_proba = 50.0

    # 5. Output
    print(f"\n--- Prediction Result ---")
    print(f"Distance: {delivery_distance_km:.2f} km")
    print(f"Estimated Traffic-Adjusted Travel Time: {estimated_travel_time_traffic_adjusted:.1f} min")
    print(f"Current Traffic Density (categorical): {traffic_density}")
    print(f"Weather: {weather_main}, temp: {temp}°C, wind_speed: {wind_speed}")
    print(f"Probability of Being Late: {prediction_proba:.2f}%")
    if prediction_proba > 60:
        print("Conclusion: High risk of late delivery (Predicted Late).")
    elif prediction_proba > 40:
        print("Conclusion: Moderate risk of late delivery.")
    else:
        print("Conclusion: Low risk of late delivery.")

        import requests

def get_location_name(lat, lon):
    """
    Returns a human-readable location name from latitude and longitude.
    """
    try:
        url = "https://nominatim.openstreetmap.org/reverse"
        params = {
            'lat': lat,
            'lon': lon,
            'format': 'json',
            'addressdetails': 1
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            return data.get('display_name', f"{lat},{lon}")
        else:
            return f"{lat},{lon}"
    except Exception as e:
        print(f"Reverse geocoding error: {e}")
        return f"{lat},{lon}"

# Example
lat, lon = 28.6139, 77.2090
print(get_location_name(lat, lon))


# --- Cell 7: Execute the Full Pipeline ---

# Step 1 & 2: Get Data and Features
df_train, feature_list = get_or_create_dataset()
print("\n" + "="*50 + "\n")

# Step 3: Preprocessing Setup
preprocessor = create_preprocessing_pipeline(feature_list)
X_train, X_test, y_train, y_test = perform_preprocessing(df_train, feature_list, preprocessor)
print("\n" + "="*50 + "\n")

# Step 4: Train, Evaluate, and Save Model
trained_pipeline = train_and_evaluate_model(X_train, X_test, y_train, y_test, preprocessor)
print("\n" + "="*50 + "\n")

# Step 5: Real-time Prediction Demonstration (using the trained model)
create_real_time_dashboard(trained_pipeline, feature_list)

# If running in Colab and you want to download the model file:
try:
    from google.colab import files
    files.download('model.pkl')
except Exception:
    # Not running in Colab, ignore.
    pass


Found existing installation: scikit-learn 1.7.2
Uninstalling scikit-learn-1.7.2:
  Successfully uninstalled scikit-learn-1.7.2
  Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.5 MB)


Environment setup check completed. Required libraries are installed.
28.6139,77.209
... Loading or creating simulated historical dataset ...
Dataset created with 500 samples. Target variable 'is_late' distribution:
is_late
1    0.5
0    0.5
Name: proportion, dtype: float64


Preprocessing pipeline (Scaling + One-Hot Encoding) created.
Data split: Train size: 400, Test size: 100


... Training XGBoost model ...
Training completed in 0.08 seconds.

--- Model Evaluation (Test Set) ---
              precision    recall  f1-score   support

           0       0.57      0.58      0.57        50
           1       0.57      0.56      0.57        50

    accuracy                           0.57       100
   macro avg       0.57      0.57      0.57       100
weighted avg       0.57      0.57      0.57       100

ROC AUC Score: 0.5356
Model saved as 'late_delivery_predictor_model.pkl'.



--- Conceptual Streamlit Dashboard Code (omitted) ---
To run the full dashboard, save the Streamlit app separ

/tmp/ipython-input-1874625372.py:172: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'Order_Placed_Time': pd.to_datetime(pd.date_range('2025-01-01', periods=500, freq='4H')),
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:26:57] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Weather fetched successfully: Clear, 20.15°C
Live traffic data successfully retrieved from HERE API.

--- Prediction Result ---
Distance: 14.79 km
Estimated Traffic-Adjusted Travel Time: 52.5 min
Current Traffic Density (categorical): Jam
Weather: Clear, temp: 20.15°C, wind_speed: 1.78
Probability of Being Late: 82.36%
Conclusion: High risk of late delivery (Predicted Late).


In [1]:
requirements = """
scikit-learn==1.7.2
xgboost
pandas
numpy
joblib
requests
streamlit
"""

with open("requirements.txt", "w") as f:
    f.write(requirements)

print("requirements.txt created successfully!")

requirements.txt created successfully!


In [2]:
!pip install streamlit


In [1]:
%%writefile app.py
import streamlit as st
MODEL_PATH = "late_delivery_predictor_model.pkl"

try:
    model_pipeline = joblib.load(MODEL_PATH)
except Exception as e:
    st.error(f"❌ Model could NOT be loaded!\nError: {e}")
    st.stop()

st.success("Model loaded successfully ✔")

Overwriting app.py


In [3]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import requests
from datetime import datetime
import math
# Load Model
# -------------------------
MODEL_PATH = "late_delivery_predictor_model.pkl"

try:
    model_pipeline = joblib.load(MODEL_PATH)
except Exception as e:
    st.error(f"❌ Model could NOT be loaded!\nError: {e}")
    st.stop()

st.success("Model loaded successfully ✔")


# -------------------------
# API Keys (YOUR keys)
# -------------------------
WEATHER_API_KEY = "YOUR_OPENWEATHERMAP_KEY"
HERE_API_KEY = "YOUR_HERE_API_KEY"


# -------------------------
# Helper Functions
# -------------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    a = (
        np.sin((lat2 - lat1) / 2) ** 2
        + np.cos(lat1) * np.cos(lat2) * np.sin((lon2 - lon1) / 2) ** 2
    )
    return R * 2 * np.arcsin(np.sqrt(a))


def fetch_realtime_weather(lat, lon):
    if WEATHER_API_KEY == "YOUR_OPENWEATHERMAP_KEY":
        return 25.0, "Clear", 4.0

    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={WEATHER_API_KEY}&units=metric"
        r = requests.get(url, timeout=5)
        data = r.json()

        temp = data["main"]["temp"]
        weather_main = data["weather"][0]["main"]
        wind = data["wind"]["speed"]

        return temp, weather_main, wind

    except:
        return 25.0, "Clear", 4.0


def fetch_live_traffic_time(rest_lat, rest_lon, del_lat, del_lon):
    if HERE_API_KEY == "YOUR_HERE_API_KEY":
        return None, None

    try:
        url = "https://router.hereapi.com/v8/routes"
        params = {
            "transportMode": "car",
            "origin": f"{rest_lat},{rest_lon}",
            "destination": f"{del_lat},{del_lon}",
            "routingMode": "fast",
            "trafficMode": "realtime",
            "return": "summary",
            "apiKey": HERE_API_KEY,
        }
        r = requests.get(url, params=params, timeout=8)
        data = r.json()
        summary = data["routes"][0]["sections"][0]["summary"]

        return summary["duration"] / 60, summary["baseDuration"] / 60

    except:
        return None, None
        import requests

def get_location_name(lat, lon):
    """
    Returns a human-readable location name from latitude and longitude.
    """
    try:
        url = "https://nominatim.openstreetmap.org/reverse"
        params = {
            'lat': lat,
            'lon': lon,
            'format': 'json',
            'addressdetails': 1
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            return data.get('display_name', f"{lat},{lon}")
        else:
            return f"{lat},{lon}"
    except Exception as e:
        print(f"Reverse geocoding error: {e}")
        return f"{lat},{lon}"

# Example
lat, lon = 28.6139, 77.2090
print(get_location_name(lat, lon))


# -------------------------
# Streamlit UI
# -------------------------
st.title("🚚 Late Delivery Prediction Dashboard")
st.markdown("Enter details to predict whether a delivery will be **late or on time**.")

col1, col2 = st.columns(2)

with col1:
    rest_lat = st.number_input("Restaurant Latitude", 28.50, 28.80, 28.60)
    rest_lon = st.number_input("Restaurant Longitude", 77.10, 77.40, 77.20)
    prep_time = st.slider("Preparation Time (min)", 5, 60, 20)
    rest_rating = st.slider("Restaurant Rating", 1.0, 5.0, 4.2)

with col2:
    del_lat = st.number_input("Delivery Latitude", 28.50, 28.80, 28.70)
    del_lon = st.number_input("Delivery Longitude", 77.10, 77.40, 77.25)
    del_rating = st.slider("Delivery Person Rating", 1.0, 5.0, 4.9)


if st.button("Predict Late Delivery 🚀"):

    distance = haversine(rest_lat, rest_lon, del_lat, del_lon)
    now = datetime.now()
    hr = now.hour
    sin_hr = math.sin(2 * math.pi * hr / 24)
    cos_hr = math.cos(2 * math.pi * hr / 24)

    temp, weather_main, wind = fetch_realtime_weather(del_lat, del_lon)

    t_live, t_base = fetch_live_traffic_time(rest_lat, rest_lon, del_lat, del_lon)

    # --- Fallback traffic simulation ---
    if t_live is None:
        if 17 <= hr <= 21:
            traffic_density = "Jam"
        elif 12 <= hr <= 14:
            traffic_density = "High"
        elif 8 <= hr <= 10:
            traffic_density = "Medium"
        else:
            traffic_density = "Low"
    else:
        ratio = t_live / t_base
        if ratio >= 1.5:
            traffic_density = "Jam"
        elif ratio >= 1.25:
            traffic_density = "High"
        elif ratio >= 1.05:
            traffic_density = "Medium"
        else:
            traffic_density = "Low"

    # --- Model Input ---
    input_df = pd.DataFrame({
        "delivery_distance_km": [distance],
        "preparation_time_min": [prep_time],
        "restaurant_rating": [rest_rating],
        "delivery_person_rating": [del_rating],
        "Road_Traffic_Density": [traffic_density],
        "Weather_Condition": [weather_main],
        "sin_hour": [sin_hr],
        "cos_hour": [cos_hr],
        "current_temp_c": [temp],
    })

    try:
        late_prob = model_pipeline.predict_proba(input_df)[0][1] * 100
        st.subheader(f"📊 Late Delivery Probability: **{late_prob:.2f}%**")

        if late_prob > 60:
            st.error("❗ High Risk of Being LATE")
        elif late_prob > 40:
            st.warning("⚠️ Medium Risk of Late")
        else:
            st.success("✅ Low Risk of Late Delivery")

    except Exception as e:
        st.error(f"Prediction Failed: {e}")


Writing app.py


In [4]:
from google.colab import files
files.download("requirements.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
import sklearn
sklearn.__version__

'1.6.1'

In [36]:
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.7.2

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 49.1 MB/s eta 0:00:00
